<a href="https://colab.research.google.com/github/heokwon/Data-Handling-for-segmentation/blob/main/Balanced_Labels0823.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## label - 0:500 , 1:500 , 2:500 , 3:500, 4:447 , 5:996
total - 3443

### Load Dataset

In [ ]:
## 데이터셋 불러오기
## 512x512 convert data
## 2048x2048 -> 512x512 crop dataset
## 2048x2048 -> 256x256 crop -> 512x512 resize dataset

!gdown 
!gdown 
!gdown 
!unzip /content/ResizeCrop_256to512_Multi_8763.zip
!unzip /content/crop_size_512x512_cropsize4_mask_exist_multi_dataset_220820.zip
!unzip /content/Multi_512x512_MMSeg.zip

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Make directory

In [ ]:
## label data를 옮길 폴더만들기
!mkdir /content/ResizeCrop_256to512_Multi/labels_src
!mkdir /content/crop_size_512x512_16_mask_exist/labels_src

In [ ]:
## convert dataset -> content에 train,label 폴더생성 -> 보기 편하게 옮겨담을 폴더 생성 후 옮기기
!mkdir Convert_512x512
!mv /content/train /content/images
!mv /content/masks /content/labels
!mv /content/images /content/Convert_512x512
!mv /content/labels /content/Convert_512x512
!mkdir /content/Convert_512x512/labels_src

In [ ]:
## convert image data = png확장자, 우리가 사용할 데이터셋 = jpg확장자
## png가 닮긴 폴더명 변경
## jpg로 바꿔서 옮겨줄 images폴더 생성
!mv /content/Convert_512x512/images /content/Convert_512x512/images_png
!mkdir /content/Convert_512x512/images 

In [ ]:
## png2jpg : png_path경로의 png파일들을 jpg로 변경 후
## jpg_des 경로에 저장
png_path='/content/Convert_512x512/images_png/' # 소스경로
jpg_des='/content/Convert_512x512/images/' # 목적지
for png in os.listdir(png_path):
    png_dir=os.path.join(png_path,png)
    png2jpg=Image.open(png_dir)
    png2jpg.save(jpg_des+png[:-4]+'.jpg')

In [ ]:
len(os.listdir(jpg_des)) # 개수확인

2895

In [ ]:
## png폴더 삭제(사용안할파일들)
!rm -rf /content/Convert_512x512/images_png

### label > 0 ~ 4 : 500개씩 뽑아오기

In [ ]:
## 뽑아온 데이터셋을 저장할 경로 생성
!mkdir Multi_512x512_BalancedLabel_minsize
!mkdir /content/Multi_512x512_BalancedLabel_minsize/images
!mkdir /content/Multi_512x512_BalancedLabel_minsize/labels

In [ ]:
## 소스경로에서 원하는 라벨값의 마스크 이미지를
## 원하는 개수만큼 뽑아와 복사
import random
import shutil
# src : label 소스경로
# class_num : label값(mask array)
# end_point : 뽑아내고싶은 이미지 개수
def move_labels(src,des,class_num,end_point):
    random.shuffle(os.listdir(src)) # 리스트 내의 원소들을 랜덤하게 섞음
    temp = 0
    for filename in tqdm(os.listdir(src)):
        label_dir=os.path.join(src,filename)
        label=Image.open(label_dir)
        label=np.array(label)
        if label.max() == class_num: # label의 array 최대값이 class_num과 같으면
            shutil.copy(src+filename,des+filename) # class_num 값의 label 파일들을 des경로에 저장
            temp += 1 # 파일 하나 저장할 때 마다 temp +1
        if temp == end_point: # temp가 end_point가 되면
            break # 중단

In [ ]:
la_src='/content/crop_size_512x512_16_mask_exist/masks/'
la_des='/content/crop_size_512x512_16_mask_exist/labels_src/'
bg_src='/content/Convert_512x512/labels/'
bg_des='/content/Convert_512x512/labels_src/'
move_labels(bg_src,bg_des,0,500)
move_labels(la_src,la_des,1,500)
move_labels(la_src,la_des,2,500)
move_labels(la_src,la_des,3,500)
move_labels(la_src,la_des,4,500)
move_labels(la_src,la_des,5,500)

  0%|          | 0/2895 [00:00<?, ?it/s]

  0%|          | 0/3112 [00:00<?, ?it/s]

  0%|          | 0/3112 [00:00<?, ?it/s]

  0%|          | 0/3112 [00:00<?, ?it/s]

  0%|          | 0/3112 [00:00<?, ?it/s]

  0%|          | 0/3112 [00:00<?, ?it/s]

### label 5 (lung) : 3개의 데이터셋 소스에서 전부 가져와 합치기
lung의 데이터셋이 너무 적어 라벨별 편차가 너무 큼

In [ ]:
## 소스 경로 내의 label파일중 원하는 label값을 가진 데이터 가져오기
def move_img(src,des,class_num):
    for mask in tqdm(os.listdir(src)):
        mask_dir=os.path.join(src,mask)
        move_mask=Image.open(mask_dir)
        move_mask=np.array(move_mask)
        if move_mask.max() == class_num :
            shutil.copy(src+mask,des+mask)

In [ ]:
path1='/content/Convert_512x512/labels/'
path2='/content/ResizeCrop_256to512_Multi/labels/'
des1='/content/Convert_512x512/labels_src/'
des2='/content/ResizeCrop_256to512_Multi/labels_src/'

In [ ]:
move_img(path1,des1,5)
move_img(path2,des2,5)

  0%|          | 0/2895 [00:00<?, ?it/s]

  0%|          | 0/8763 [00:00<?, ?it/s]

In [ ]:
len(os.listdir(la_des)),len(os.listdir(des1)),len(os.listdir(des2))

(2219, 675, 549)

### label data와 이름이 같은 image data 옮기기

In [ ]:
## 뽑아온 label data와 매칭되는 image file 옮기기
def moveimgfile(image_src,masks_src,image_des):
    for img_, mask in zip(os.listdir(image_src),os.listdir(masks_src)):
        name = mask.split('.')[0] # id.png -> 구분자(.)기준으로 나눠줌, id=[0] png[1], 
        image_name = name + '.jpg' # imagefile name = 'id.jpg' 으로 설정
        img_ = image_name # label과 매칭되는 id의 이미지들 지정
        shutil.copy(image_src+img_,image_des+img_)

In [ ]:
image_src1='/content/Convert_512x512/images/'
image_src2='/content/ResizeCrop_256to512_Multi/images/'
image_src3='/content/crop_size_512x512_16_mask_exist/images/'
mask_src1='/content/Convert_512x512/labels_src/'
mask_src2='/content/ResizeCrop_256to512_Multi/labels_src/'
mask_src3='/content/crop_size_512x512_16_mask_exist/labels_src/'
image_des='/content/Multi_512x512_BalancedLabel_minsize/images/'

In [ ]:
moveimgfile(image_src1,mask_src1,image_des)
moveimgfile(image_src2,mask_src2,image_des)
moveimgfile(image_src3,mask_src3,image_des)

In [ ]:
## image file 개수확인
len(os.listdir(image_des))

3443

### 라벨별로 500개씩 뽑아온 마스크 데이터셋 이동

In [ ]:
def move_file(src,des):
    for mask_file in tqdm(os.listdir(src)):
        shutil.copy(src+mask_file,des+mask_file)

In [ ]:
src_path1='/content/Convert_512x512/labels_src/'
src_path2='/content/ResizeCrop_256to512_Multi/labels_src/'
src_path3='/content/crop_size_512x512_16_mask_exist/labels_src/'
labels_des='/content/Multi_512x512_BalancedLabel_minsize/labels/'

move_file(src_path1,labels_des)
move_file(src_path2,labels_des)
move_file(src_path3,labels_des)

  0%|          | 0/675 [00:00<?, ?it/s]

  0%|          | 0/549 [00:00<?, ?it/s]

  0%|          | 0/2219 [00:00<?, ?it/s]

In [ ]:
## image와 label data 개수 일치하는지 확인
len(os.listdir(image_des)),len(os.listdir(labels_des))

(3443, 3443)

### 만든 Dataset 압축 후 드라이브로 경로이동

In [ ]:
!zip Multi_512x512_BalancedLabel_minsize.zip -r ./Multi_512x512_BalancedLabel_minsize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/Multi_512x512_BalancedLabel_minsize.zip /content/drive/MyDrive/MMSeg_data

### binary

In [ ]:
!gdown 

Downloading...
From: https://drive.google.com/uc?id=11aCGZSG3dZrYVgglYFktlXlAAsmiJr8s
To: /content/Multi_512x512_BalancedLabel_minsize.zip
100% 173M/173M [00:01<00:00, 130MB/s]


In [ ]:
!unzip /content/Multi_512x512_BalancedLabel_minsize.zip

In [ ]:
import os
import numpy as np
from PIL import Image

masks_path='/content/Multi_512x512_BalancedLabel_minsize/labels/'
for mask in os.listdir(masks_path):
    mask_dir=os.path.join(masks_path,mask)
    binary=Image.open(mask_dir)
    binary=np.array(binary)
    binary=np.where(binary>0, 1, 0)
  
    binary=Image.fromarray(binary.astype(np.uint8))
    binary.save(masks_path+mask)

In [ ]:
!mv /content/Multi_512x512_BalancedLabel_minsize /content/Binary_512x512_BalancedLabel_minsize

In [ ]:
!zip Binary_512x512_BalancedLabel_minsize.zip -r ./Binary_512x512_BalancedLabel_minsize

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: Binary_512x512_BalancedLabel_minsize/images/29296_003.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/10703_003.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/18445_007.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/2696_013.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/15499_011.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/29610_004.jpg (deflated 14%)
  adding: Binary_512x512_BalancedLabel_minsize/images/22718_007.jpg (deflated 0%)
  adding: Binary_512x512_BalancedLabel_minsize/images/2793_0006.jpg (deflated 12%)
  adding: Binary_512x512_BalancedLabel_minsize/images/2696_008.jpg (deflated 1%)
  adding: Binary_512x512_BalancedLabel_minsize/images/31571_001.jpg (deflated 1%)
  adding: Binary_512x512_BalancedLabel_minsize/images/7359_512_1536.jpg (deflated 5%)
  adding: Binary_512x512_BalancedLabel_minsize/images/5086

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/Binary_512x512_BalancedLabel_minsize.zip /content/drive/MyDrive/Kaggle